# 📊 SmartPortfolio AI - Interactive Dashboard

## Notebook 06: Portfolio Analytics & Insights

**Goal**: Comprehensive visualization and analysis of your AI-powered investment strategy

**What You'll See**:
- 📈 **Portfolio Overview**: Holdings, allocation, and performance
- 🎯 **Stock Analysis**: Individual stock metrics and rankings
- 🧠 **ML Model Insights**: Prediction accuracy and feature importance
- 📊 **Risk Dashboard**: Volatility, correlations, and risk metrics
- 💰 **Performance Tracking**: Returns, Sharpe ratio, and benchmark comparison
- 🔮 **Future Outlook**: Monte Carlo projections and recommendations

**Interactive Features**:
- Plotly charts (hover, zoom, filter)
- Sector breakdowns
- Stock deep-dives
- Risk heat maps

**Last Updated**: 2025-10-03 16:32

---


## 📚 Step 1: Import Libraries


In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

# Database
from dotenv import load_dotenv
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))
load_dotenv(project_root / '.env')
from utils.database_connector import DatabaseConnector

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
%matplotlib inline

# Set styling
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries loaded successfully!")
print(f"📁 Project root: {project_root}")


✅ Libraries loaded successfully!
📁 Project root: /Users/RiyanshiKedia/Documents/GitHub/smartportfolio/SmartPortfolio


## 📂 Step 2: Load Portfolio Data


In [2]:
# Find latest output files
data_dir = project_root / 'data' / 'processed'

# Load portfolio allocation
portfolio_files = sorted(data_dir.glob('portfolio_allocation_*.csv'), reverse=True)
if not portfolio_files:
    raise FileNotFoundError("No portfolio files found. Run Notebook 04 first!")

portfolio_file = portfolio_files[0]
portfolio = pd.read_csv(portfolio_file)
portfolio = portfolio[portfolio['weight'] > 0.001].copy()

print(f"✅ Portfolio loaded: {portfolio_file.name}")
print(f"   Stocks: {len(portfolio)}")
print(f"   Total Value: ${portfolio['dollar_amount'].sum():,.2f}")

# Load portfolio summary
summary_files = sorted(data_dir.glob('portfolio_summary_*.json'), reverse=True)
if summary_files:
    with open(summary_files[0], 'r') as f:
        portfolio_summary = json.load(f)
    print(f"✅ Portfolio summary loaded")
else:
    portfolio_summary = None
    print("⚠️  Portfolio summary not found")

# Load ML predictions
ml_files = sorted(data_dir.glob('ml_predictions_*.csv'), reverse=True)
if ml_files:
    ml_predictions = pd.read_csv(ml_files[0])
    print(f"✅ ML predictions loaded: {len(ml_predictions)} stocks")
else:
    ml_predictions = None
    print("⚠️  ML predictions not found")

# Load performance simulation
sim_files = sorted(data_dir.glob('performance_simulation_*.json'), reverse=True)
if sim_files:
    with open(sim_files[0], 'r') as f:
        performance = json.load(f)
    print(f"✅ Performance simulation loaded")
else:
    performance = None
    print("⚠️  Performance simulation not found")

print("\n✅ All data loaded!")


✅ Portfolio loaded: portfolio_allocation_20251003_1611.csv
   Stocks: 50
   Total Value: $5,000.00
✅ Portfolio summary loaded
✅ ML predictions loaded: 503 stocks
✅ Performance simulation loaded

✅ All data loaded!


## 📊 Step 3: Portfolio Overview Dashboard


In [3]:
# Create summary statistics
total_value = portfolio['dollar_amount'].sum()
num_stocks = len(portfolio)
top_holding = portfolio.iloc[0]

print("=" * 80)
print("💼 PORTFOLIO OVERVIEW")
print("=" * 80)
print(f"\n💰 Total Portfolio Value: ${total_value:,.2f}")
print(f"📈 Number of Holdings: {num_stocks}")
print(f"🏆 Top Holding: {top_holding['ticker']} ({top_holding['company_name']})")
print(f"   Weight: {top_holding['weight']*100:.2f}%")
print(f"   Value: ${top_holding['dollar_amount']:,.2f}")

# Sector breakdown
if 'sector' in portfolio.columns:
    print(f"\n🏢 Sector Diversification:")
    sector_alloc = portfolio.groupby('sector')['dollar_amount'].sum().sort_values(ascending=False)
    for sector, amount in sector_alloc.items():
        pct = (amount / total_value) * 100
        print(f"   {sector:<25} ${amount:>8,.0f} ({pct:>5.1f}%)")

# Concentration metrics
top_5_weight = portfolio.head(5)['weight'].sum()
top_10_weight = portfolio.head(10)['weight'].sum()
print(f"\n📊 Concentration:")
print(f"   Top 5 holdings: {top_5_weight*100:.1f}%")
print(f"   Top 10 holdings: {top_10_weight*100:.1f}%")

print("=" * 80)


💼 PORTFOLIO OVERVIEW

💰 Total Portfolio Value: $5,000.00
📈 Number of Holdings: 50
🏆 Top Holding: NEE (NextEra Energy)
   Weight: 10.00%
   Value: $500.00

🏢 Sector Diversification:
   Financials                $   1,266 ( 25.3%)
   Consumer Staples          $   1,006 ( 20.1%)
   Health Care               $     600 ( 12.0%)
   Utilities                 $     550 ( 11.0%)
   Communication Services    $     400 (  8.0%)
   Information Technology    $     400 (  8.0%)
   Industrials               $     328 (  6.6%)
   Consumer Discretionary    $     200 (  4.0%)
   Materials                 $     150 (  3.0%)
   Energy                    $     100 (  2.0%)

📊 Concentration:
   Top 5 holdings: 47.3%
   Top 10 holdings: 60.0%


## 📈 Step 4: Portfolio Allocation Visualizations


In [4]:
# Create interactive allocation charts
from plotly.subplots import make_subplots

# 1. Top Holdings Bar Chart
fig1 = go.Figure()

top_15 = portfolio.head(15).copy()
top_15 = top_15.sort_values('weight', ascending=True)

fig1.add_trace(go.Bar(
    y=top_15['ticker'],
    x=top_15['weight'] * 100,
    orientation='h',
    text=[f"{w*100:.1f}%" for w in top_15['weight']],
    textposition='auto',
    marker=dict(
        color=top_15['weight'] * 100,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Weight (%)")
    ),
    hovertemplate='<b>%{y}</b><br>Weight: %{x:.2f}%<br><extra></extra>'
))

fig1.update_layout(
    title='Top 15 Portfolio Holdings',
    xaxis_title='Portfolio Weight (%)',
    yaxis_title='Ticker',
    height=600,
    showlegend=False,
    template='plotly_white'
)

fig1.show()

# 2. Sector Allocation Pie Chart
if 'sector' in portfolio.columns:
    sector_data = portfolio.groupby('sector').agg({
        'dollar_amount': 'sum',
        'ticker': 'count'
    }).reset_index()
    sector_data.columns = ['sector', 'value', 'count']
    
    fig2 = go.Figure(data=[go.Pie(
        labels=sector_data['sector'],
        values=sector_data['value'],
        hole=0.4,
        textinfo='label+percent',
        hovertemplate='<b>%{label}</b><br>Value: $%{value:,.0f}<br>Percent: %{percent}<extra></extra>'
    )])
    
    fig2.update_layout(
        title='Portfolio Allocation by Sector',
        height=500,
        template='plotly_white'
    )
    
    fig2.show()

# 3. Treemap of all holdings
fig3 = px.treemap(
    portfolio.head(30),
    path=['sector', 'ticker'] if 'sector' in portfolio.columns else ['ticker'],
    values='dollar_amount',
    color='weight',
    hover_data={'company_name': True, 'weight': ':.2%', 'shares': True},
    color_continuous_scale='RdYlGn',
    title='Portfolio Treemap (Top 30 Holdings)'
)

fig3.update_layout(height=600)
fig3.show()

print("✅ Interactive charts displayed!")


✅ Interactive charts displayed!


## 🏆 Step 5: Stock Rankings & Analysis


In [8]:
# Connect to database for additional metrics
db = DatabaseConnector()

# Get latest prices and calculate recent performance
tickers = portfolio['ticker'].tolist()
tickers_str = "', '".join(tickers)

query = f"""
WITH LatestPrices AS (
    SELECT 
        ticker,
        [close] as current_price,
        date as [current_date],
        LAG([close], 5) OVER (PARTITION BY ticker ORDER BY date) as price_1w,
        LAG([close], 20) OVER (PARTITION BY ticker ORDER BY date) as price_1m,
        LAG([close], 60) OVER (PARTITION BY ticker ORDER BY date) as price_3m,
        volume,
        ROW_NUMBER() OVER (PARTITION BY ticker ORDER BY date DESC) as rn
    FROM market.daily_prices
    WHERE ticker IN ('{tickers_str}')
)
SELECT 
    ticker,
    current_price,
    [current_date],
    (current_price - price_1w) / NULLIF(price_1w, 0) * 100 as return_1w,
    (current_price - price_1m) / NULLIF(price_1m, 0) * 100 as return_1m,
    (current_price - price_3m) / NULLIF(price_3m, 0) * 100 as return_3m,
    volume
FROM LatestPrices
WHERE rn = 1
ORDER BY ticker
"""

prices_df = db.execute_query(query)

# Convert return columns to numeric (handle NaN/None)
for col in ['return_1w', 'return_1m', 'return_3m']:
    if col in prices_df.columns:
        prices_df[col] = pd.to_numeric(prices_df[col], errors='coerce')

# Merge with portfolio
portfolio_analysis = portfolio.merge(prices_df, on='ticker', how='left')

# Add ML predictions if available
if ml_predictions is not None:
    ml_subset = ml_predictions[['ticker', 'pred_return_1M', 'pred_return_3M']].copy()
    portfolio_analysis = portfolio_analysis.merge(ml_subset, on='ticker', how='left')

# Display top performers
print("=" * 80)
print("🏆 TOP PERFORMERS (Past Month)")
print("=" * 80)
top_performers = portfolio_analysis.nlargest(10, 'return_1m')
print(f"\n{'Rank':<6} {'Ticker':<8} {'Company':<30} {'1M Return':<12} {'Weight':<10}")
print("-" * 80)
for idx, row in enumerate(top_performers.itertuples(), 1):
    print(f"{idx:<6} {row.ticker:<8} {row.company_name[:28]:<30} {row.return_1m:>10.2f}% {row.weight*100:>8.2f}%")

print("\n" + "=" * 80)
print("📉 WEAKEST PERFORMERS (Past Month)")
print("=" * 80)
weak_performers = portfolio_analysis.nsmallest(10, 'return_1m')
print(f"\n{'Rank':<6} {'Ticker':<8} {'Company':<30} {'1M Return':<12} {'Weight':<10}")
print("-" * 80)
for idx, row in enumerate(weak_performers.itertuples(), 1):
    print(f"{idx:<6} {row.ticker:<8} {row.company_name[:28]:<30} {row.return_1m:>10.2f}% {row.weight*100:>8.2f}%")

print("=" * 80)





2025-10-03 16:35:34,042 - INFO - Attempting connection with pymssql...
2025-10-03 16:35:34,044 - INFO - ✅ Database connection established
2025-10-03 16:35:34,206 - INFO - ✅ Query executed successfully, returned 50 rows


🏆 TOP PERFORMERS (Past Month)

Rank   Ticker   Company                        1M Return    Weight    
--------------------------------------------------------------------------------
1      WDC      Western Digital                     52.03%     1.00%
2      HOOD     Robinhood Markets                   38.01%     1.00%
3      TSLA     Tesla, Inc.                         37.53%     1.00%
4      SMCI     Supermicro                          30.65%     1.00%
5      ORCL     Oracle Corporation                  29.34%     1.00%
6      CNC      Centene Corporation                 18.90%    10.00%
7      MRNA     Moderna                             14.76%     1.00%
8      NEM      Newmont                             14.34%     1.00%
9      AVGO     Broadcom                            10.44%     1.00%
10     NEE      NextEra Energy                       9.83%    10.00%

📉 WEAKEST PERFORMERS (Past Month)

Rank   Ticker   Company                        1M Return    Weight    
--------------------

## 📊 Step 6: Historical Performance Analysis


In [11]:
# Load historical prices for portfolio stocks
query_hist = f"""
SELECT 
    ticker,
    date,
    [close] as price
FROM market.daily_prices
WHERE ticker IN ('{tickers_str}')
  AND date >= DATEADD(day, -180, (SELECT MAX(date) FROM market.daily_prices))
ORDER BY ticker, date
"""

hist_prices = db.execute_query(query_hist)
hist_prices['date'] = pd.to_datetime(hist_prices['date'])

# Convert price to float (handle Decimal type from SQL Server)
hist_prices['price'] = pd.to_numeric(hist_prices['price'], errors='coerce')

# Pivot to wide format
price_matrix = hist_prices.pivot(index='date', columns='ticker', values='price')

# Calculate portfolio value over time
weights = portfolio.set_index('ticker')['weight']
weights = weights.reindex(price_matrix.columns, fill_value=0)

# Normalize prices to returns
returns = price_matrix.pct_change().fillna(0)
portfolio_returns = (returns * weights.values).sum(axis=1)
portfolio_cumulative = (1 + portfolio_returns).cumprod()

# Load SPY for comparison if available
query_spy = """
SELECT date, [close] as price
FROM market.daily_prices
WHERE ticker = 'SPY'
  AND date >= DATEADD(day, -180, (SELECT MAX(date) FROM market.daily_prices))
ORDER BY date
"""

try:
    spy_prices = db.execute_query(query_spy)
    if len(spy_prices) > 0:
        spy_prices['date'] = pd.to_datetime(spy_prices['date'])
        spy_prices['price'] = pd.to_numeric(spy_prices['price'], errors='coerce')
        spy_prices = spy_prices.set_index('date')
        spy_returns = spy_prices['price'].pct_change().fillna(0)
        
        # Align dates
        common_dates = portfolio_cumulative.index.intersection(spy_returns.index)
        spy_returns_aligned = spy_returns.loc[common_dates]
        spy_cumulative = (1 + spy_returns_aligned).cumprod()
        
        has_spy = True
        print("✅ SPY benchmark data loaded")
    else:
        has_spy = False
        print("⚠️  No SPY data available")
except:
    has_spy = False
    print("⚠️  Could not load SPY data")

# Create interactive performance chart
fig = go.Figure()

# Portfolio line
fig.add_trace(go.Scatter(
    x=portfolio_cumulative.index,
    y=(portfolio_cumulative - 1) * 100,
    mode='lines',
    name='Portfolio',
    line=dict(color='blue', width=3),
    hovertemplate='Date: %{x}<br>Return: %{y:.2f}%<extra></extra>'
))

# SPY line if available
if has_spy:
    fig.add_trace(go.Scatter(
        x=spy_cumulative.index,
        y=(spy_cumulative - 1) * 100,
        mode='lines',
        name='S&P 500 (SPY)',
        line=dict(color='orange', width=2, dash='dash'),
        hovertemplate='Date: %{x}<br>Return: %{y:.2f}%<extra></extra>'
    ))

fig.add_hline(y=0, line_dash="dot", line_color="gray", opacity=0.5)

fig.update_layout(
    title='Portfolio Performance (6 Months)',
    xaxis_title='Date',
    yaxis_title='Cumulative Return (%)',
    height=500,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

# Calculate metrics
total_return = (portfolio_cumulative.iloc[-1] - 1) * 100
annual_volatility = portfolio_returns.std() * np.sqrt(252) * 100
sharpe = (portfolio_returns.mean() * 252 - 0.045) / (portfolio_returns.std() * np.sqrt(252))

print(f"\n📊 Performance Metrics (6-Month Period):")
print(f"   Total Return: {total_return:+.2f}%")
print(f"   Annualized Volatility: {annual_volatility:.2f}%")
print(f"   Sharpe Ratio: {sharpe:.3f}")

if has_spy:
    spy_total = (spy_cumulative.iloc[-1] - 1) * 100
    print(f"   SPY Total Return: {spy_total:+.2f}%")
    print(f"   Outperformance: {total_return - spy_total:+.2f}%")

db.close()


2025-10-03 16:42:08,997 - INFO - ✅ Query executed successfully, returned 6150 rows
2025-10-03 16:42:09,008 - INFO - ✅ Query executed successfully, returned 124 rows


✅ SPY benchmark data loaded


2025-10-03 16:42:09,019 - INFO - 🔌 Database connection closed



📊 Performance Metrics (6-Month Period):
   Total Return: +25.09%
   Annualized Volatility: 17.89%
   Sharpe Ratio: 2.401
   SPY Total Return: +33.29%
   Outperformance: -8.20%


## 🧠 Step 7: ML Model Insights


In [12]:
if ml_predictions is not None:
    # Merge predictions with portfolio
    ml_portfolio = portfolio.merge(
        ml_predictions[['ticker', 'pred_return_1M', 'pred_return_3M', 'rank']],
        on='ticker',
        how='left'
    )
    
    # Create prediction scatter plot
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=ml_portfolio['pred_return_1M'],
        y=ml_portfolio['pred_return_3M'],
        mode='markers',
        marker=dict(
            size=ml_portfolio['weight'] * 1000,  # Size by weight
            color=ml_portfolio['weight'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Weight"),
            line=dict(width=1, color='white')
        ),
        text=ml_portfolio['ticker'],
        hovertemplate='<b>%{text}</b><br>1M Pred: %{x:.2f}%<br>3M Pred: %{y:.2f}%<extra></extra>'
    ))
    
    fig.update_layout(
        title='ML Predictions: 1-Month vs 3-Month Returns',
        xaxis_title='Predicted 1-Month Return (%)',
        yaxis_title='Predicted 3-Month Return (%)',
        height=500,
        template='plotly_white'
    )
    
    fig.show()
    
    # Top ML picks
    print("=" * 80)
    print("🧠 TOP ML-PREDICTED STOCKS IN PORTFOLIO")
    print("=" * 80)
    top_ml = ml_portfolio.nlargest(10, 'pred_return_3M')
    print(f"\n{'Ticker':<8} {'Company':<30} {'1M Pred':<12} {'3M Pred':<12} {'Weight':<10}")
    print("-" * 80)
    for row in top_ml.itertuples():
        print(f"{row.ticker:<8} {row.company_name[:28]:<30} {row.pred_return_1M:>10.2f}% {row.pred_return_3M:>10.2f}% {row.weight*100:>8.2f}%")
    print("=" * 80)
else:
    print("⚠️  ML predictions not available. Run Notebook 03 first.")




KeyError: "['ml_rank'] not in index"

## ⚠️ Step 8: Risk Analysis Dashboard


In [13]:
# Correlation matrix
correlation_matrix = returns[portfolio.head(20)['ticker'].tolist()].corr()

# Create correlation heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    colorscale='RdBu',
    zmid=0,
    text=correlation_matrix.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 8},
    hovertemplate='%{x} vs %{y}<br>Correlation: %{z:.3f}<extra></extra>'
))

fig.update_layout(
    title='Stock Correlation Matrix (Top 20 Holdings)',
    height=700,
    width=700,
    template='plotly_white'
)

fig.show()

# Volatility analysis
volatilities = returns[portfolio['ticker'].tolist()].std() * np.sqrt(252) * 100
vol_df = pd.DataFrame({
    'ticker': volatilities.index,
    'volatility': volatilities.values
})
vol_df = vol_df.merge(portfolio[['ticker', 'weight', 'company_name']], on='ticker')
vol_df = vol_df.sort_values('volatility', ascending=False)

# Volatility chart
fig = go.Figure()

top_20_vol = vol_df.head(20).sort_values('volatility', ascending=True)

fig.add_trace(go.Bar(
    y=top_20_vol['ticker'],
    x=top_20_vol['volatility'],
    orientation='h',
    marker=dict(
        color=top_20_vol['volatility'],
        colorscale='Reds',
        showscale=True,
        colorbar=dict(title="Volatility (%)")
    ),
    text=[f"{v:.1f}%" for v in top_20_vol['volatility']],
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>Annual Volatility: %{x:.2f}%<extra></extra>'
))

fig.update_layout(
    title='Stock Volatility Rankings (Top 20 Most Volatile)',
    xaxis_title='Annualized Volatility (%)',
    yaxis_title='Ticker',
    height=600,
    template='plotly_white'
)

fig.show()

# Risk/Return scatter
avg_returns = returns[portfolio['ticker'].tolist()].mean() * 252 * 100
risk_return = pd.DataFrame({
    'ticker': avg_returns.index,
    'return': avg_returns.values,
    'volatility': volatilities.values
})
risk_return = risk_return.merge(portfolio[['ticker', 'weight']], on='ticker')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=risk_return['volatility'],
    y=risk_return['return'],
    mode='markers',
    marker=dict(
        size=risk_return['weight'] * 1000,
        color=risk_return['return'],
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title="Return (%)"),
        line=dict(width=1, color='white')
    ),
    text=risk_return['ticker'],
    hovertemplate='<b>%{text}</b><br>Volatility: %{x:.1f}%<br>Return: %{y:.1f}%<extra></extra>'
))

fig.update_layout(
    title='Risk-Return Profile',
    xaxis_title='Volatility (Annual %)',
    yaxis_title='Return (Annual %)',
    height=500,
    template='plotly_white'
)

fig.show()

print("✅ Risk analysis complete!")


✅ Risk analysis complete!


## 🔮 Step 9: Future Projections


In [14]:
if performance is not None:
    mc = performance.get('monte_carlo', {})
    
    print("=" * 80)
    print("🔮 FUTURE OUTLOOK (1-Year Monte Carlo Simulation)")
    print("=" * 80)
    
    if mc:
        initial = 5000  # From config
        mean_final = mc.get('mean_final_value', 0)
        median_final = mc.get('median_final_value', 0)
        prob_loss = mc.get('probability_loss', 0)
        var_95 = mc.get('var_95', 0)
        
        print(f"\n💰 Expected Outcomes:")
        print(f"   Starting Value: ${initial:,.2f}")
        print(f"   Mean Final Value: ${mean_final:,.2f}")
        print(f"   Expected Return: {(mean_final/initial - 1)*100:+.2f}%")
        print(f"   Median Final Value: ${median_final:,.2f}")
        
        print(f"\n⚠️  Risk Metrics:")
        print(f"   Probability of Loss: {prob_loss*100:.1f}%")
        print(f"   Probability of Gain: {(1-prob_loss)*100:.1f}%")
        print(f"   Value at Risk (95%): ${abs(var_95):,.2f}")
        
        print(f"\n📊 Confidence Intervals:")
        # These would come from percentiles in simulation
        print(f"   5th Percentile: Worst case scenario")
        print(f"   50th Percentile: Most likely outcome")
        print(f"   95th Percentile: Best case scenario")
        
        # Investment recommendation
        print(f"\n✅ RECOMMENDATION:")
        backtest = performance.get('backtest', {})
        sharpe = backtest.get('sharpe_ratio', 0)
        
        if sharpe > 1.0 and prob_loss < 0.3:
            print(f"   🟢 STRONG BUY")
            print(f"   • High risk-adjusted returns (Sharpe: {sharpe:.2f})")
            print(f"   • Low probability of loss ({prob_loss*100:.1f}%)")
            print(f"   • Diversified portfolio with {len(portfolio)} holdings")
        elif sharpe > 0.5 and prob_loss < 0.4:
            print(f"   🟡 MODERATE BUY")
            print(f"   • Decent risk-adjusted returns (Sharpe: {sharpe:.2f})")
            print(f"   • Acceptable risk level")
        else:
            print(f"   🔴 CAUTION")
            print(f"   • Review risk tolerance carefully")
            print(f"   • Consider rebalancing")
    
    print("=" * 80)
else:
    print("⚠️  Performance simulation not available. Run Notebook 05 first.")


🔮 FUTURE OUTLOOK (1-Year Monte Carlo Simulation)

💰 Expected Outcomes:
   Starting Value: $5,000.00
   Mean Final Value: $6,179.73
   Expected Return: +23.59%
   Median Final Value: $6,061.42

⚠️  Risk Metrics:
   Probability of Loss: 16.5%
   Probability of Gain: 83.5%
   Value at Risk (95%): $643.79

📊 Confidence Intervals:
   5th Percentile: Worst case scenario
   50th Percentile: Most likely outcome
   95th Percentile: Best case scenario

✅ RECOMMENDATION:
   🟡 MODERATE BUY
   • Decent risk-adjusted returns (Sharpe: 0.98)
   • Acceptable risk level


## 📋 Step 10: Complete Portfolio Summary


In [15]:
# Create comprehensive summary table
summary_table = portfolio[['ticker', 'company_name', 'sector', 'weight', 
                           'dollar_amount', 'shares']].copy()

# Add price data if available
if 'current_price' in portfolio_analysis.columns:
    summary_table = summary_table.merge(
        portfolio_analysis[['ticker', 'current_price', 'return_1w', 'return_1m', 'return_3m']],
        on='ticker',
        how='left'
    )

# Add ML predictions if available
if ml_predictions is not None and 'pred_return_3M' in ml_predictions.columns:
    summary_table = summary_table.merge(
        ml_predictions[['ticker', 'pred_return_3M']],
        on='ticker',
        how='left'
    )

# Format for display
summary_table['weight'] = summary_table['weight'].apply(lambda x: f"{x*100:.2f}%")
summary_table['dollar_amount'] = summary_table['dollar_amount'].apply(lambda x: f"${x:,.0f}")

print("=" * 80)
print("📊 COMPLETE PORTFOLIO HOLDINGS")
print("=" * 80)
print(f"\nShowing all {len(summary_table)} positions\n")

# Display full table
display(summary_table)

# Export option
export_path = data_dir / f'dashboard_summary_{datetime.now().strftime("%Y%m%d_%H%M")}.csv'
summary_table.to_csv(export_path, index=False)
print(f"\n✅ Summary exported to: {export_path.name}")



📊 COMPLETE PORTFOLIO HOLDINGS

Showing all 50 positions



,ticker,company_name,sector,weight,dollar_amount,shares,pred_return_3M
0,NEE,NextEra Energy,Utilities,10.00%,$500,6,0.086290
1,MDLZ,Mondelez International,Consumer Staples,10.00%,$500,7,0.045024
2,CNC,Centene Corporation,Health Care,10.00%,$500,13,0.054620
3,WFC,Wells Fargo,Financials,10.00%,$500,6,0.064142
4,C,Citigroup,Financials,7.31%,$366,3,0.072323
5,PG,Procter & Gamble,Consumer Staples,6.13%,$306,2,0.079875
6,BA,Boeing,Industrials,3.56%,$178,0,0.053574
7,SLB,Schlumberger,Energy,1.00%,$50,1,0.056253
8,FCX,Freeport-McMoRan,Materials,1.00%,$50,1,0.188617
9,HOOD,Robinhood Markets,Financials,1.00%,$50,0,0.216255



✅ Summary exported to: dashboard_summary_20251003_1644.csv


## 📝 Step 11: Next Actions & Recommendations


In [16]:
print("=" * 80)
print("📋 RECOMMENDED NEXT ACTIONS")
print("=" * 80)

print(f"\n🔄 Portfolio Maintenance:")
print(f"   1. Monitor positions weekly")
print(f"   2. Rebalance monthly (if drift > 5%)")
print(f"   3. Review stop-losses for volatile holdings")
print(f"   4. Check for corporate actions (dividends, splits)")

print(f"\n🔍 Areas to Monitor:")

# Check concentration
if top_5_weight > 0.5:
    print(f"   ⚠️  HIGH CONCENTRATION: Top 5 = {top_5_weight*100:.1f}%")
    print(f"      Consider reducing position sizes")

# Check sector exposure
if 'sector' in portfolio.columns:
    max_sector = portfolio.groupby('sector')['weight'].sum().max()
    if max_sector > 0.35:
        max_sector_name = portfolio.groupby('sector')['weight'].sum().idxmax()
        print(f"   ⚠️  SECTOR RISK: {max_sector_name} = {max_sector*100:.1f}%")
        print(f"      Consider sector diversification")

# Check weak performers
weak_count = len(portfolio_analysis[portfolio_analysis['return_1m'] < -10])
if weak_count > 0:
    print(f"   ⚠️  {weak_count} stocks down >10% this month")
    print(f"      Review fundamentals and consider trimming")

print(f"\n📅 Regular Updates:")
print(f"   • Daily: Check major news and market events")
print(f"   • Weekly: Review performance vs benchmark")
print(f"   • Monthly: Rebalance if needed")
print(f"   • Quarterly: Re-run ML predictions and optimization")

print(f"\n🔄 Refresh Schedule:")
print(f"   • Notebook 01: Monthly (update price data)")
print(f"   • Notebook 02: Monthly (re-screen stocks)")
print(f"   • Notebook 03: Quarterly (retrain ML models)")
print(f"   • Notebook 04: Monthly (re-optimize allocation)")
print(f"   • Notebook 05: Monthly (update backtest)")
print(f"   • Notebook 06: Weekly (review dashboard)")

print("=" * 80)
print("✅ Dashboard complete!")
print("=" * 80)


📋 RECOMMENDED NEXT ACTIONS

🔄 Portfolio Maintenance:
   1. Monitor positions weekly
   2. Rebalance monthly (if drift > 5%)
   3. Review stop-losses for volatile holdings
   4. Check for corporate actions (dividends, splits)

🔍 Areas to Monitor:
   ⚠️  1 stocks down >10% this month
      Review fundamentals and consider trimming

📅 Regular Updates:
   • Daily: Check major news and market events
   • Weekly: Review performance vs benchmark
   • Monthly: Rebalance if needed
   • Quarterly: Re-run ML predictions and optimization

🔄 Refresh Schedule:
   • Notebook 01: Monthly (update price data)
   • Notebook 02: Monthly (re-screen stocks)
   • Notebook 03: Quarterly (retrain ML models)
   • Notebook 04: Monthly (re-optimize allocation)
   • Notebook 05: Monthly (update backtest)
   • Notebook 06: Weekly (review dashboard)
✅ Dashboard complete!


## 💾 Step 12: Export Dashboard Data


In [17]:
# Create comprehensive export package
timestamp = datetime.now().strftime('%Y%m%d_%H%M')

# Export portfolio with all metrics
full_export = portfolio.copy()

if 'current_price' in portfolio_analysis.columns:
    full_export = full_export.merge(
        portfolio_analysis[['ticker', 'current_price', 'return_1w', 'return_1m', 'return_3m']],
        on='ticker',
        how='left'
    )

if ml_predictions is not None:
    full_export = full_export.merge(
        ml_predictions[['ticker', 'pred_return_1M', 'pred_return_3M', 'rank']],
        on='ticker',
        how='left'
    )

export_file = data_dir / f'dashboard_full_{timestamp}.csv'
full_export.to_csv(export_file, index=False)

print(f"✅ Full dashboard data exported:")
print(f"   {export_file.name}")
print(f"   {len(full_export)} stocks with complete metrics")

# Create JSON summary
dashboard_summary = {
    'generated_at': timestamp,
    'portfolio': {
        'total_value': float(portfolio['dollar_amount'].sum()),
        'num_holdings': len(portfolio),
        'top_holding': portfolio.iloc[0]['ticker'],
    },
    'performance': {
        'total_return': float(total_return),
        'volatility': float(annual_volatility),
        'sharpe_ratio': float(sharpe)
    }
}

if performance:
    dashboard_summary['simulation'] = performance.get('monte_carlo', {})

json_file = data_dir / f'dashboard_summary_{timestamp}.json'
with open(json_file, 'w') as f:
    json.dump(dashboard_summary, f, indent=2)

print(f"\n✅ Summary JSON exported:")
print(f"   {json_file.name}")

print(f"\n📊 Dashboard session complete!")




✅ Full dashboard data exported:
   dashboard_full_20251003_1644.csv
   50 stocks with complete metrics

✅ Summary JSON exported:
   dashboard_summary_20251003_1644.json

📊 Dashboard session complete!
